In [46]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import sys
import numpy as np
import pickle
from sklearn.utils import shuffle

In [47]:
dummy_data = pd.read_excel('DataSets\Dummy_data_testing.xlsx')
dummy_data_cleaned = dummy_data.drop(
    ['nameOrig', 'nameDest', 'OrigCountry', 'DestCountry'], axis=1)
dummy_data_cleaned = pd.get_dummies(dummy_data_cleaned)

In [48]:
y_dummy = dummy_data_cleaned.loc[:, 'isFraud']
X_dummy = dummy_data_cleaned.drop(['isFraud'], axis=1)

In [49]:
classifier = RandomForestClassifier(n_estimators=50)
filename = 'TM_Model_Kushagra.h5'
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_dummy)

In [50]:
y_predected_dummy = y_pred
y_predected_dummy = list(y_predected_dummy)
dummy_data = dummy_data.drop('isFraud', axis=1)
dummy_data['isFraud'] = y_predected_dummy

In [51]:
df_red = dummy_data[dummy_data['isFraud'] == 1]
df_red = df_red.drop('isFraud', axis=1)
df_red['Flag'] = 'Red'
# Saving the Red File To The CSV Format --------
df_red.to_csv('Generated_Csv\Red_Flagged.csv')

In [52]:
df2 = pd.DataFrame(pd.read_excel('DataSets\Contries_score.xlsx'))
df2 = df2.rename(columns={'Overall score': 'score'})
df2 = df2[df2['score'] >= 6]
country_score = pd.Series(df2.score.values, index=df2.Country).to_dict()

In [53]:
dummy_data = dummy_data[dummy_data['isFraud'] == 0]
dummy_data.drop('isFraud', axis=1)

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,OrigCountry,DestCountry
0,CASH_OUT,229133.94,C905080434,15325.00,0.00,C476402209,5083.00,51513.44,AFGHANISTAN,CAPE VERDE
1,TRANSFER,311685.89,C1984094095,10835.00,0.00,C932583850,6267.00,2719172.89,HAITI,SRI LANKA
2,TRANSFER,7413.54,C1470868839,297689.58,290276.03,M1426725223,0.00,0.00,MYANMAR,PARAGUAY
3,TRANSFER,6660.33,C893523498,17710.00,11049.67,M1770070706,0.00,0.00,LAOS,BAHAMAS
4,TRANSFER,26923.42,C1108517064,11049.67,0.00,C1983747920,199404.91,95508.95,MOZAMBIQUE,TANZANIA
...,...,...,...,...,...,...,...,...,...,...
91,TRANSFER,312895.63,C1724067349,40171.00,0.00,C1651945012,0.00,1556799.98,BOSNIA-HERZEGOVINA,ANDORRA
93,CASH_IN,367834.87,C1023460014,902685.56,1270520.43,C1609562230,4883037.95,4515203.08,MARSHALL ISLANDS,MYANMAR
94,CASH_IN,287937.99,C1716876539,1270520.43,1558458.42,C1423992712,9977117.37,9689179.38,HONDURAS,UZBEKISTAN
95,CASH_IN,37360.00,C1052245880,1558458.42,1595818.41,C790722099,88544.15,51184.15,MOROCCO,ECUADOR


In [54]:
orange_flag = {}
green_flag = {}
for i in dummy_data.index:
    if dummy_data['OrigCountry'][i] in country_score.keys() or dummy_data['DestCountry'][i] in country_score.keys():
        if ((dummy_data['amount'][i] > 100000)):
            orange_flag.update({dummy_data['nameOrig'][i]: 'orange'})
    else:
        green_flag.update({dummy_data['nameOrig'][i]: 'green'})

In [55]:
orange = pd.DataFrame(orange_flag.items(), columns=['nameOrig', 'Flag'])
green = pd.DataFrame(green_flag.items(), columns=['nameOrig', 'Flag'])
df_orange = dummy_data.merge(orange, how="inner")
df_orange.to_csv('Generated_Csv\ORANGE_FLAGGED.csv')
df_green = dummy_data.merge(green, how="inner")
df_green.to_csv('Generated_Csv\Green_Flagged.csv')

In [56]:
df_orange.to_json('client\src\data\Organge_Flagged.json', orient='records')
df_red.to_json('client\src\data\Red_Flagged.json', orient='records')
df_green.to_json('client\src\data\Green_Flagged.json', orient='records')

In [59]:
send_back1 = df_orange
send_back2=df_green
send_back3=df_red
print('Orange\n\n {} \n\n Green \n\n {} \n\n Red \n\n {} '.format(send_back1,send_back2,send_back3),flush=True)
sys.stdout.flush()

Orange

         type      amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0   CASH_OUT   229133.94   C905080434       15325.00            0.00   
1   TRANSFER   311685.89  C1984094095       10835.00            0.00   
2   TRANSFER   211075.91  C1540894701           0.00            0.00   
3   TRANSFER   100677.86   C203149502           0.00            0.00   
4   TRANSFER   473499.51  C1198197478           0.00            0.00   
5   TRANSFER  1538200.39   C476579021           0.00            0.00   
6   TRANSFER  2421578.09   C106297322           0.00            0.00   
7   TRANSFER  1349670.68  C1419332030           0.00            0.00   
8   TRANSFER   314403.14  C1262110193           0.00            0.00   
9   TRANSFER  1457213.54   C396918327           0.00            0.00   
10  TRANSFER   180368.84  C1055601039           0.00            0.00   
11  TRANSFER   445038.71   C547441493           0.00            0.00   
12  TRANSFER  1123206.66   C967677821           0.00   

In [58]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
import base64
from email import encoders
import smtplib
red_file="Generated_Csv\Red_Flagged.csv"
sender_address = 'amlteam.lscg@gmail.com'
sender_pass = 'lscg2020'                                                         
recipients=['kushagra.tandon@zohomail.eu','kushagra.25@zohomail.eu','akshit.gupta2001@zohomail.eu','tarun.tiwari2000@zohomail.eu']
message_content="""<!DOCTYPE html>
<html>
    <body>
        <h2 style="color:Red;">ALERT !!! A Fraud Has Been Detected by LS&CG</h2>
        <h5 style="color:SlateGray;">The Money Laundering Cases Has Been Detected.</h5>
        <img src="cid:Fraudimage" width='400' height=200>
        <h5 style="color:Black;">The Data is Being Attached Along With The Email. Please Look Into The Transaction & Notify On Server</h5>
        <h5 style="color:Black;">Update The Final Result On akshit_web_link.com</h5>
        <h6 style="color:Red;">This Is A System Generated Email Please Do Not Reply To This E-Mail. For More Information Contact LS&CG</h6>
        
    </body>
</html>
"""
message = MIMEMultipart()
message['From'] = sender_address
message['To'] = ", ".join(recipients)
message['Subject'] = 'FRAUD DETECTECTION!!!'                                                                                           
                                                                                                
message.attach(MIMEText(message_content, 'html'))
attach_file_name = red_file                                                            
attach_file = open(attach_file_name, 'rb')                                      
payload = MIMEBase('application', 'octate-stream')
payload.set_payload((attach_file).read())
encoders.encode_base64(payload) 
                                                                                                
payload.add_header('Content-Decomposition', 'attachment', filename=attach_file_name)
message.attach(payload)

fp = open('DataSets/fraud.jpg', 'rb')
image = MIMEImage(fp.read())
fp.close()
image.add_header('Content-ID', '<Fraudimage>')
message.attach(image)
                                                                                                 
session = smtplib.SMTP('smtp.gmail.com', 587)                                   
session.starttls()                                                               
session.login(sender_address, sender_pass)                                       
text = message.as_string()
session.sendmail(sender_address, recipients, text)
session.quit()
print('The Mail Has Been Send Successfully')

orange_file="Generated_Csv\ORANGE_FLAGGED.csv"
sender_address = 'amlteam.lscg@gmail.com'
sender_pass = 'lscg2020'                                                         
recipients=['tarun.tiwari2000@zohomail.eu','kushagra.tandon@zohomail.eu','kushagra.25@zohomail.eu','akshit.gupta2001@zohomail.eu']
message_content="""<!DOCTYPE html>
<html>
    <body>
        <h2 style="color:Red;">ALERT !!! A Potential Fraud Has Been Detected by LS&CG</h2>
        <h5 style="color:SlateGray;">The Money Laundering Cases Has Been Detected.</h5>
        <img src="cid:Fraudimage" width='400' height=200>
        <h5 style="color:Black;">The Data is Being Attached Along With The Email. Please Look Into The Transaction & Notify On Server</h5>
        <h5 style="color:Black;">Update The Final Result On akshit_web_link.com</h5>
        <h6 style="color:Red;">This Is A System Generated Email Please Do Not Reply To This E-Mail. For More Information Contact LS&CG</h6>
    </body>
</html>
"""
message = MIMEMultipart()
message['From'] = sender_address
message['To'] = ", ".join(recipients)
message['Subject'] = 'FRAUD DETECTECTION!!!'                                                                                                                                                               
message.attach(MIMEText(message_content, 'html'))
attach_file_name = red_file                                                            
attach_file = open(attach_file_name, 'rb')                                      
payload = MIMEBase('application', 'octate-stream')
payload.set_payload((attach_file).read())
encoders.encode_base64(payload)                                                                          
payload.add_header('Content-Decomposition', 'attachment', filename=attach_file_name)
message.attach(payload)
fp = open('DataSets/fraud.jpg', 'rb')
image = MIMEImage(fp.read())
fp.close()
image.add_header('Content-ID', '<Fraudimage>')
message.attach(image)                                                                                  
session = smtplib.SMTP('smtp.gmail.com', 587)                                   
session.starttls()                                                               
session.login(sender_address, sender_pass)                                       
text = message.as_string()
session.sendmail(sender_address, recipients, text)
session.quit()

The Mail Has Been Send Successfully


(221, b'2.0.0 closing connection n4sm956873pgv.89 - gsmtp')